### Часть первая - обкачка aot ( http://aot.ru/demo/morph.html )

In [122]:
import requests
import json
import copy
from tqdm import tqdm
import random
from collections import defaultdict as dd

In [123]:
accent_dict = dd(set)
word_dict = dd(set)
nom_accent_dict = dd(set)
nom_word_dict = dd(set)
ins_accent_dict = dd(set)
ins_word_dict = dd(set)
dat_accent_dict = dd(set)
dat_word_dict = dd(set)
acc_accent_dict = dd(set)
acc_word_dict = dd(set)

Функция, делающая запрос сайту и возвращающая json со всеми вариантами слова и их парадигмами (с проставленным ударением, если слово найдено в словаре)

In [124]:
def get_aot_json(word):
    url = 'http://63.250.59.227:8080/?'
    data = {'action': 'morph',
            'langua': 'Russian',
            'query': word,
            'withparadigms': '1',}
    html_content = requests.get(url, data).text
    return html_content

In [125]:
print(get_aot_json('кукла'))

[{"commonGrammems":"но","found":true,"morphInfo":"С жр,им,ед","paradigm":[{"formsGroups":[{"forms":[{"f":"КУ'КЛА","grm":"им"},{"f":"КУ'КЛЫ","grm":"рд"},{"f":"КУ'КЛЕ","grm":"дт"},{"f":"КУ'КЛУ","grm":"вн"},{"f":"КУ'КЛОЮ","grm":"тв"},{"f":"КУ'КЛОЙ","grm":"тв"},{"f":"КУ'КЛЕ","grm":"пр"}],"grm":"ед"},{"forms":[{"f":"КУ'КЛЫ","grm":"им"},{"f":"КУ'КОЛ","grm":"рд"},{"f":"КУ'КЛАМ","grm":"дт"},{"f":"КУ'КЛЫ","grm":"вн"},{"f":"КУ'КЛАМИ","grm":"тв"},{"f":"КУ'КЛАХ","grm":"пр"}],"grm":"мн"}],"pos":"СУЩЕСТВИТЕЛЬНОЕ жр"}],"wordForm":"КУКЛА"},{"commonGrammems":"од","found":true,"morphInfo":"С жр,им,ед","paradigm":[{"formsGroups":[{"forms":[{"f":"КУ'КЛА","grm":"им"},{"f":"КУ'КЛЫ","grm":"рд"},{"f":"КУ'КЛЕ","grm":"дт"},{"f":"КУ'КЛУ","grm":"вн"},{"f":"КУ'КЛОЮ","grm":"тв"},{"f":"КУ'КЛОЙ","grm":"тв"},{"f":"КУ'КЛЕ","grm":"пр"}],"grm":"ед"},{"forms":[{"f":"КУ'КЛЫ","grm":"им"},{"f":"КУ'КОЛ","grm":"рд"},{"f":"КУ'КЛАМ","grm":"дт"},{"f":"КУ'КОЛ","grm":"вн"},{"f":"КУ'КЛАМИ","grm":"тв"},{"f":"КУ'КЛАХ","grm":"пр"}],"gr

Функция, достающая список слов, при необходимости только конкретного(ых) падежа(ей)

In [126]:
def get_wordlist(word, cases='им,рд,дт,вн,тв,пр'):
    a = get_aot_json(word)
    b = json.loads(a)
    forms_list = []
    cases = cases.split(',')
    for paradigm in b[0]['paradigm']:
        for formsGroup in paradigm['formsGroups']:
            for form in formsGroup['forms']:
                if form['grm'] in cases:
                    forms_list.append(form['f'])
    return forms_list

In [127]:
print(get_wordlist('шнурок'))
print(get_wordlist('шнурок', cases='дт,им'))

["ШНУРО'К", "ШНУРКА'", "ШНУРКУ'", "ШНУРО'К", "ШНУРКО'М", "ШНУРКЕ'", "ШНУРКИ'", "ШНУРКО'В", "ШНУРКА'М", "ШНУРКИ'", "ШНУРКА'МИ", "ШНУРКА'Х"]
["ШНУРО'К", "ШНУРКУ'", "ШНУРКИ'", "ШНУРКА'М"]


In [128]:
print(get_wordlist('ужин'))
print(get_wordlist('шнурок', cases='дт,им'))

["УЖИ'Н", "У'ЖИН", "УЖИ'НА", "У'ЖИНА", "УЖИ'НУ", "У'ЖИНУ", "УЖИ'Н", "У'ЖИН", "УЖИ'НОМ", "У'ЖИНОМ", "УЖИ'НЕ", "У'ЖИНЕ", "У'ЖИНЫ", "УЖИ'НЫ", "УЖИ'НОВ", "У'ЖИНОВ", "УЖИ'НАМ", "У'ЖИНАМ", "УЖИ'НЫ", "У'ЖИНЫ", "УЖИ'НАМИ", "У'ЖИНАМИ", "УЖИ'НАХ", "У'ЖИНАХ"]
["ШНУРО'К", "ШНУРКУ'", "ШНУРКИ'", "ШНУРКА'М"]


Функция, добавляющая словоформы в словари:
- в accent_dict ключи - кортежи из количества слогов и номера ударного слога, значения - слова (уже без знака ударения)
- в word_dict наоборот, ключи - слова, значения - кортежи из количества слогов и номера ударного слога

количество слогов считается по количеству гласных, ударный слог определяется количеством гласных перед апострофом (обозначением ударения)

In [129]:
def def_accents(wordlist, dict_accents, dict_words):
    vowels = ['А', 'Е', 'И', 'О', 'У', 'Ы', 'Э', 'Ю', 'Я']
    for word in wordlist:
        parts = word.split("'")
        if len(parts) == 2:
            vown = 0
            for ch in parts[0]:
                if ch in vowels:
                    vown += 1
            accent_num = copy.deepcopy(vown)
            for ch in parts[1]:
                if ch in vowels:
                    vown += 1
            syllable_num = copy.deepcopy(vown)
            new_word = ''.join(parts)
            key_tuple = (syllable_num, accent_num)
            dict_accents[key_tuple].add(new_word)
            dict_words[new_word].add(key_tuple)
    return

Вот как это выглядит:

In [130]:
def_accents(get_wordlist('корова'), accent_dict, word_dict)
def_accents(get_wordlist('коза'), accent_dict, word_dict)

In [131]:
accent_dict

defaultdict(set,
            {(3, 2): {'КОЗОЮ',
              'КОРОВА',
              'КОРОВАМ',
              'КОРОВАХ',
              'КОРОВЕ',
              'КОРОВОЙ',
              'КОРОВУ',
              'КОРОВЫ'},
             (4, 2): {'КОРОВАМИ', 'КОРОВОЮ'},
             (2, 2): {'КОЗА', 'КОЗЕ', 'КОЗОЙ', 'КОЗУ', 'КОЗЫ', 'КОРОВ'},
             (2, 1): {'КОЗАМ', 'КОЗАХ', 'КОЗЫ'},
             (1, 1): {'КОЗ'},
             (3, 1): {'КОЗАМИ'}})

In [132]:
word_dict

defaultdict(set,
            {'КОРОВА': {(3, 2)},
             'КОРОВЫ': {(3, 2)},
             'КОРОВЕ': {(3, 2)},
             'КОРОВУ': {(3, 2)},
             'КОРОВОЮ': {(4, 2)},
             'КОРОВОЙ': {(3, 2)},
             'КОРОВ': {(2, 2)},
             'КОРОВАМ': {(3, 2)},
             'КОРОВАМИ': {(4, 2)},
             'КОРОВАХ': {(3, 2)},
             'КОЗА': {(2, 2)},
             'КОЗЫ': {(2, 1), (2, 2)},
             'КОЗЕ': {(2, 2)},
             'КОЗУ': {(2, 2)},
             'КОЗОЮ': {(3, 2)},
             'КОЗОЙ': {(2, 2)},
             'КОЗ': {(1, 1)},
             'КОЗАМ': {(2, 1)},
             'КОЗАМИ': {(3, 1)},
             'КОЗАХ': {(2, 1)}})

Возьмём наш список слов и преобразуем с помощью этих функций в словарь:

In [133]:
with open('codenames_biggest_one.txt', 'r', encoding='utf-8') as file:
    text = file.read()
    print(text[2200:2300])

 вод­ка, во­ен­ный, вождь, воз­вра­ще­ние, воз­дух, воз­мож­ность, воз­раст, во­ин, вой, вой­на, вок


In [134]:
words = text
words = words.replace('­','').split(', ')
print('слов', len(words))

слов 2272


In [50]:
for word in tqdm(words[-10:]):
    def_accents(get_wordlist(word), accent_dict, word_dict)
    def_accents(get_wordlist(word, 'им'), nom_accent_dict, nom_word_dict)
    def_accents(get_wordlist(word, 'тв'), ins_accent_dict, ins_word_dict)
    def_accents(get_wordlist(word, 'дт'), dat_accent_dict, dat_word_dict)
    def_accents(get_wordlist(word, 'вн'), acc_accent_dict, acc_word_dict)

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [01:05<00:00,  1.53it/s]


In [184]:
print(len(dict_list))

10


In [135]:
import pickle

with open('codenames_big_dicts.pickle', 'wb') as handle:
    pickle.dump([accent_dict, word_dict, nom_accent_dict, nom_word_dict, ins_accent_dict, ins_word_dict, dat_accent_dict, dat_word_dict, acc_accent_dict, acc_word_dict], handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('codenames_big_dicts.pickle', 'rb') as handle:
    dict_list = pickle.load(handle)

In [136]:
accent_dict, word_dict, nom_accent_dict, nom_word_dict, ins_accent_dict, ins_word_dict, dat_accent_dict, dat_word_dict, acc_accent_dict, acc_word_dict = dict_list

### Часть вторая - генерация стихотворения

Функция, создающая строку, написанную двухсложным метром с ударением на всех чётных или нечётных слогах:
- подбираются все возможные комбинации количества слогов и номера ударного слога для данного момента строки
- слова из них объединяются в один список
- выбирается случайное
- слово добавляется в строку, количество слогов вычитается

In [137]:
def poetry(n, ac, dict_accents, dict_words):
    stroka = []
    str_len = ac #(уже заполнено)
    full = n+ac #(длина итоговой строки)
    
    while True:
        if full - str_len == 3:
            word = random.choice(list(dict_accents[(3,str_len%2 + 2)]))
            stroka.append(word.lower())
            str_len += 3
            break
        if full - str_len == 1:
            word = random.choice(list(dict_accents[(1, 1)]))
            stroka.append(word.lower())
            str_len += 1
            break
        else:
            #keys = (любое число от 1 до 4, набор ударений: или [1,3] или [2,4]str_len%2)
            keys = []
            if str_len%2 == 0:
                accents = [2, 4]
            else:
                accents = [1, 3]
            for i in range(1,min(4, full-str_len+1)):
                if (full-str_len-i) != 1:
                    for j in accents:
                        if not (full-str_len == 4 and i == 4 and j in [1,2]):
                            keys.append((i,j))
            w_list = []
            for key in keys:
                w_list.extend(list(dict_accents[key]))
            word = random.choice(w_list)
            stroka.append(word.lower())
            str_len += list(dict_words[word])[0][0]
            if full-str_len == 1:
                print(stroka)
        if str_len == full:
            break
    return ' '.join(stroka)
        

Генерация, полностью из случайных слов

In [173]:
def random_poem(dict1, dict2):
    poem = '\n'.join([poetry(9, 2, dict1, dict2),
                       poetry(8, 2, dict1, dict2),
                       poetry(9, 2, dict1, dict2),
                       poetry(8, 2, dict1, dict2)])
    return poem

In [145]:
print(random_poem(accent_dict, word_dict))

смертях скандал беседам штампы
зрачки война весны пятков
резервы личности обходом
вилков натур клокам сетям


In [170]:
print(random_poem(nom_accent_dict, nom_word_dict))

платок беседы нерв аккорды
цветы просмотры миллион
телега перья мальчик штуки
супруги жулик красота


Генерация со вспомогательными строками и закрепленным падежом

In [146]:
def template1(nom_dict1, nom_dict2):
    poem = '\n'.join(['заходит в дом олег под вечер',
                      'а там ' + poetry(6, 2, nom_dict1, nom_dict2),
                      poetry(9, 2, nom_dict1, nom_dict2),
                      'и дом вообще-то не его'])
    return poem

In [147]:
print(template1(nom_accent_dict, nom_word_dict))

заходит в дом олег под вечер
а там печать сады сеанс
мечта вокзалы гром презренья
и дом вообще-то не его


In [148]:
def template2(ins_dict1, ins_dict2):
    poem = '\n'.join(['оксана мчится ' + poetry(4, 1, ins_dict1, ins_dict2),
                      poetry(8, 2, ins_dict1, ins_dict2),
                      poetry(9, 2, ins_dict1, ins_dict2),
                      'но магазин уже закрыт'])
    return poem

In [150]:
print(template2(ins_accent_dict, ins_word_dict))

оксана мчится детством бомбой
цепочкой взрослою тайгой
уроком болями бутылкой
но магазин уже закрыт


In [151]:
def template3(nom_dict1, nom_dict2):
    poem = '\n'.join(['у роботов другие чувства',
                      'не ' + poetry(1, 1, nom_dict1, nom_dict2)
                      + ' не ' + poetry(2, 1, nom_dict1, nom_dict2)
                      + ' не ' + poetry(2, 2, nom_dict1, nom_dict2),
                      'а ' + poetry(8, 1, nom_dict1, nom_dict2),
                      poetry(5, 2, nom_dict1, nom_dict2) + ' и ' + poetry(2, 2, nom_dict1, nom_dict2)])
    return poem

In [152]:
print(template3(nom_accent_dict, nom_word_dict))

у роботов другие чувства
не плоть не принцип не перрон
а опыты йогурт советы
икона буква и прием


In [153]:
def template4(dat_dict1, acc_dict1):
    poem = '\n'.join([random.choice(list(dat_dict1[(3, 2)])).lower() + ' ' +
                      random.choice(list(acc_dict1[(3, 1)])).lower() + ' и ' +
                      random.choice(list(acc_dict1[(2, 1)])).lower(),
                      random.choice(list(dat_dict1[(2, 2)])).lower() + ' ' +
                      random.choice(list(acc_dict1[(4, 2)])).lower() + ' и ' +
                      random.choice(list(acc_dict1[(1, 1)])).lower(),
                      random.choice(list(dat_dict1[(3, 2)])).lower() + ' ' +
                      random.choice(list(acc_dict1[(1, 1)])).lower() + ' и ' +
                      random.choice(list(acc_dict1[(4, 3)])).lower(),
                      'а мне ' + random.choice(list(acc_dict1[(3, 2)])).lower() +
                      ' и ' + random.choice(list(acc_dict1[(2, 2)])).lower()])
    return poem

In [154]:
print(template4(dat_accent_dict, acc_accent_dict))

условьям отдыхи и ласок
плащу шампанское и рельс
усмешкам суп и сообщенья
а мне ступени и вопрос


### Часть третья - начинаем генерировать сос мыслом

Используем готовую семантическую модель:

In [24]:
import re
import gensim
import logging
import nltk.data
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from gensim.models import word2vec

import warnings
warnings.filterwarnings('ignore')

In [25]:
urllib.request.urlretrieve("http://rusvectores.org/static/models/rusvectores2/ruscorpora_mystem_cbow_300_2_2015.bin.gz", "ruscorpora_mystem_cbow_300_2_2015.bin.gz")

('ruscorpora_mystem_cbow_300_2_2015.bin.gz',
 <http.client.HTTPMessage at 0x6271610>)

In [26]:
m = 'ruscorpora_mystem_cbow_300_2_2015.bin.gz'

if m.endswith('.vec.gz'):
    model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=False)
elif m.endswith('.bin.gz'):
    model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=True)
else:
    model = gensim.models.KeyedVectors.load(m)

Функция, создающая по слову отдельный словарь из близких ему слов

In [176]:
def create_custom_dict(word):
    
    custom_accent_dict = dd(set)
    custom_word_dict = dd(set)
    custom_nom_accent_dict = dd(set)
    custom_nom_word_dict = dd(set)
    custom_ins_accent_dict = dd(set)
    custom_ins_word_dict = dd(set)
    custom_dat_accent_dict = dd(set)
    custom_dat_word_dict = dd(set)
    custom_acc_accent_dict = dd(set)
    custom_acc_word_dict = dd(set)
    
    try:
        a = model.most_similar(positive=[word], topn=200)
    except Exception:
        print('Слово не найдено(')
        return 'error'
    close_words = [word.split('_')[0] for (word, num) in a]
    for w in tqdm(close_words):
        def_accents(get_wordlist(w), custom_accent_dict, custom_word_dict)
        def_accents(get_wordlist(w, 'им'), custom_nom_accent_dict, custom_nom_word_dict)
        def_accents(get_wordlist(w, 'тв'), custom_ins_accent_dict, custom_ins_word_dict)
        def_accents(get_wordlist(w, 'дт'), custom_dat_accent_dict, custom_dat_word_dict)
        def_accents(get_wordlist(w, 'вн'), custom_acc_accent_dict, custom_acc_word_dict)
    return [custom_accent_dict, custom_word_dict, custom_nom_accent_dict, custom_nom_word_dict, custom_ins_accent_dict, custom_ins_word_dict, custom_dat_accent_dict, custom_dat_word_dict, custom_acc_accent_dict, custom_acc_word_dict]

Создадим несколько наборов на разные темы

In [91]:
music_dicts = create_custom_dict('музыка_S')

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [02:08<00:00,  1.56it/s]


In [157]:
zoo_dicts = create_custom_dict('животное_S')

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [02:21<00:00,  1.41it/s]


In [101]:
wind_dicts = create_custom_dict('ветер_S')

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [02:08<00:00,  1.56it/s]


In [108]:
robot_dicts = create_custom_dict('робот_S')
love_dicts = create_custom_dict('любовь_S')

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [02:12<00:00,  1.50it/s]


In [119]:
human_dicts = create_custom_dict('человек_S')
gift_dicts = create_custom_dict('подарок_S')

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [02:08<00:00,  1.56it/s]


Применим

In [158]:
print(random_poem(music_dicts[0], music_dicts[1]))

прелюдам вальса арф фанфарам
сюитам арфою певцам
зурнах матчишах вальсу ритму
дуэтов фуге гопаком


In [95]:
print(template1(zoo_dicts[2], zoo_dicts[3]))

заходит в дом олег под вечер
а там скотина грызуны
подсвинок пес горилла кошка
и дом вообще-то не его


In [160]:
print(template2(wind_dicts[4], wind_dicts[5]))

оксана мчится штормом мозглым
крылами бурею крылом
струенным моросью метелью
но магазин уже закрыт


Немножко изменим существующий шаблон, чтобы можно было указать два разных набора словарей

In [161]:
def template3_1(nom_dict1, nom_dict2, nom_dict3, nom_dict4):
    poem = '\n'.join(['у роботов другие чувства',
                      'не ' + poetry(1, 1, nom_dict1, nom_dict2)
                      + ' не ' + poetry(2, 1, nom_dict1, nom_dict2)
                      + ' не ' + poetry(2, 2, nom_dict1, nom_dict2),
                      'а ' + poetry(8, 1, nom_dict3, nom_dict4),
                      poetry(5, 2, nom_dict3, nom_dict4) + ' и ' + poetry(2, 2, nom_dict3, nom_dict4)])
    return poem

In [162]:
print(template3_1(love_dicts[2], love_dicts[3], robot_dicts[2], robot_dicts[3]))

у роботов другие чувства
не скорбь не дружбы не мечта
а пульты рация селектор
локатор мазер и модель


In [120]:
print(template4(human_dicts[6], gift_dicts[8]))

счастливцам весточку и снимку
вещам приветствие и брошь
туземцам дар и побрякушки
а мне наследство и калым


In [183]:
print(template4(human_dicts[6], gift_dicts[8]))

людишкам складчины и торты
парням дарящее и торт
народам дар и шоколадку
а мне гостинец и бювар
